<a href="https://colab.research.google.com/github/nhs-pycom/coding-club-Python-Data-Wrangling/blob/main/Python_DataWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**The NHSX Analytics Unit Python Coding Club Session 12**

*Created by*: Andrew Sylvester

The goal for this session is calculate the ratio of IT and intangible assets to total assets per secondary provider over the time period FY2016/17 to FY2019/20. 

The content convered to do this is:
* importing and cleaning public NHS secondary provider balance sheet data
* appending dataframes together
* manipulating the data to calculate key accounting items
* reshaping it into wide format to make it easier to read


#**1.Importing the data**
 
  Start off by importing the libraries we'll use:

In [8]:
import pandas as pd
import numpy as np

The data for FY2017/18 for Foundation Trusts is stored at this link:

https://webarchive.nationalarchives.gov.uk/ukgwa/20200327163023mp_/https:/improvement.nhs.uk/documents/3142/All_TAC_data_published_in_NHS_foundation_trusts_accounts_for_2017-18.xlsx


The Excel file has mulitple tabs, with the main data in the "All data" tab, and a mapping of provdier codes to provider names in the "List of providers" tab. We will need both to be saved in different dataframes so that we can merge them afterwards.  


In [9]:
URL_fts_1718 = "https://webarchive.nationalarchives.gov.uk/ukgwa/20200327163023mp_/https:/improvement.nhs.uk/documents/3142/All_TAC_data_published_in_NHS_foundation_trusts_accounts_for_2017-18.xlsx"
fts_1718_df = pd.read_excel(URL_fts_1718, sheet_name = "All data")
fts_1718_pcode = pd.read_excel(URL_fts_1718, sheet_name = "List of providers")

In [ ]:
# View the first rows of the accounting data

fts_1718_df.head()

In [ ]:
# View the first rows of the provider code and name mapping

fts_1718_pcode.head()

Use [pd.merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) to merge the NHS code into the balance sheet data based on the organisation name. (hint: you will was the balance sheet data to be the left dataset and do a left merge so that any providers in the mapping which are not in this years balance sheet data are not copied across) 

In [ ]:
# Task 1: merge in the provider code to the balance sheet data

fts_1718_merged = pd.merge(fts_1718_df, fts_1718_pcode, how="left", left_on = "Organisation name", right_on = "Full name of Provider")
fts_1718_merged.head()

When you try to append all years together later on, you will find that for some reason the FY2017/18 datasets have the organisation name variable name spelled "Organisation name" but the other datasets spell it "Organisation name". Likewise, the FY2017/18 have a variable "Valuenumber" which has a space in later datasets so that it is "Value number". 

To save youself some trouble later, rename these two variables in the FY2017/18 data so that they are consistent with later years' variable names. [hint: use [rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)]

In [ ]:
# Task 2: Rename the two variables to be consistent with future years variable names

fts_1718_merged = fts_1718_merged.rename({"Organisation name": "Organisation Name", "Valuenumber" : "Value number"}, axis='columns')
fts_1718_merged.head()

Another thing which becomes apparent when building the multiyear appended dataset is that there is no variable to identify the year of the data, so you need to add in a year identifier before appending all the datasets together. 

In [ ]:
# Task 3: Add in a variable to the merged dataframe to identify the year of this data

fts_1718_merged['Year'] = "FY2017/18" 

Providers which are NHS Foundation Trusts have their balance sheet data saved separately to providers with are NHS Trusts. We thus need to repeat what was done above for NHS Foundation Trusts for NHS Trusts. Furthermore, we also want to do this for Trusts and FTs for more years worth of data (FY2018/19 and FY2019/20). The urls for these files are:

Trusts in FY2017/18: 

https://webarchive.nationalarchives.gov.uk/ukgwa/20200327163023mp_/https://improvement.nhs.uk/documents/3302/All_TAC_data_published_in_NHS_trusts_accounts_for_2017-18.xlsx

Trusts in FY2018/19: 

https://webarchive.nationalarchives.gov.uk/ukgwa/20200327162132mp_/https:/improvement.nhs.uk/documents/5905/TAC_data_published_in_NHS_trusts_accounts_for_2018-19.xlsx

Foundation Trusts in FY2018/19: 

https://webarchive.nationalarchives.gov.uk/ukgwa/20200327162132mp_/https:/improvement.nhs.uk/documents/5906/TAC_data_published_in_NHS_foundation_trusts_accounts_for_2018-19.xlsx

Trusts in FY2019/20: 

https://www.england.nhs.uk/wp-content/uploads/2021/04/TAC-data-published-in-NHS-trusts-accounts-for-2019-20.xlsx

Foundation Trusts in FY2019/20: 

https://www.england.nhs.uk/wp-content/uploads/2021/04/TAC-data-published-in-NHS-foundation-trusts-accounts-for-2019-20.xlsx

The next task is to import the balance sheet and provider code mapping data from all these URLs, merge the provider code into the balance sheet data, rename "Organisation name" and "Valuenumber" where relevant, and add in a variable to each merged dataframe to identify the year. Ten points to Gryffindor if you can do this in a loop.   

In [9]:
# Task 4: Do everything above for each year & trust/FT combo

dataset_names = ["fts_1718",
                 "trust_1718", 
                 "fts_1819", 
                 "trust_1819", 
                 "fts_1920", 
                 "trust_1920"]

dataset_urls = ["https://webarchive.nationalarchives.gov.uk/ukgwa/20200327163023mp_/https:/improvement.nhs.uk/documents/3142/All_TAC_data_published_in_NHS_foundation_trusts_accounts_for_2017-18.xlsx",
                "https://webarchive.nationalarchives.gov.uk/ukgwa/20200327163023mp_/https://improvement.nhs.uk/documents/3302/All_TAC_data_published_in_NHS_trusts_accounts_for_2017-18.xlsx", 
                "https://webarchive.nationalarchives.gov.uk/ukgwa/20200327162132mp_/https:/improvement.nhs.uk/documents/5906/TAC_data_published_in_NHS_foundation_trusts_accounts_for_2018-19.xlsx", 
                "https://webarchive.nationalarchives.gov.uk/ukgwa/20200327162132mp_/https:/improvement.nhs.uk/documents/5905/TAC_data_published_in_NHS_trusts_accounts_for_2018-19.xlsx", 
                "https://www.england.nhs.uk/wp-content/uploads/2021/04/TAC-data-published-in-NHS-foundation-trusts-accounts-for-2019-20.xlsx", 
                "https://www.england.nhs.uk/wp-content/uploads/2021/04/TAC-data-published-in-NHS-trusts-accounts-for-2019-20.xlsx"]

years = ["FY2017/18", 
         "FY2017/18", 
         "FY2018/19", 
         "FY2018/19", 
         "FY2019/20", 
         "FY2019/20"]

df_dict = {}

for i in range(0,6):
  URL =  dataset_urls[i]
  df_dict[dataset_names[i]+"_df"] = pd.read_excel(URL, sheet_name = "All data").rename({"Organisation name": "Organisation Name", "Valuenumber" : "Value number"}, axis='columns')
  df_dict[dataset_names[i]+"_pcode"] = pd.read_excel(URL, sheet_name = "List of providers")
  df_dict[dataset_names[i]+"_merged"] = pd.merge(df_dict[dataset_names[i]+"_df"], df_dict[dataset_names[i]+"_pcode"], how="left", left_on = "Organisation Name", right_on = "Full name of Provider")
  df_dict[dataset_names[i]+"_merged"]['Year'] = years[i]


In [ ]:
df_dict['trust_1920_merged'].head()


# **2. Appending dataframes**

The merged dataframes for each combination of trust/FT and year need to be appended into a single aggregate dataframe. [Hint: use [concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)]


In [10]:
# Task 5: build an aggreate dataframe including all the merged dataframes for each year and trust/FT combo
frames = [df_dict['fts_1718_merged'], 
          df_dict['trust_1718_merged'], 
          df_dict['fts_1819_merged'], 
          df_dict['trust_1819_merged'], 
          df_dict['fts_1920_merged'], 
          df_dict['trust_1920_merged']]
aggregate_df = pd.concat(frames)

Play with the data to make sure it has worked.

In [ ]:
aggregate_df[["Year", "Value number"]].groupby("Year").mean()


We will only need a subset fo the variables for the analysis below, so might as well clean up the aggregate dataframe at this stage. [hint: use [drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)]

In [16]:
# Task 6: Drop the variables in the Cols list below
# Cols = ["Region","Full name of Provider", "Authorisation date", "Comments "]

cols = ["Region","Full name of Provider", "Authorisation date", "Comments "]
aggregate_df = aggregate_df.drop(cols, axis=1)

# **3. Calculate key accounting items**

The balance sheet data contains many different line items of data. The key variables we need for this analysis are:

*   Opening Net Book Value ("NBV") of total assets
*   Closing NBV of total assets
*   Opening NBV of IT and intengible assets
*   Closing NBV of IT and intangible assets

To do this requires some knowledge of the accounting coding, so I have been kind enough not to make this step a task! It's worth having a look at the [.loc[ ]](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) documentation.  

In [17]:
# Create a new variable called Account_item to identify key rows of data
aggregate_df.loc[(aggregate_df['MainCode'] == 'A14CY01') & (aggregate_df['SubCode'] == 'PPE0190'), 'Account_item'] = 'Total NBV closing value'
aggregate_df.loc[(aggregate_df['MainCode'] == 'A14CY01') & (aggregate_df['SubCode'] == 'PPE0010'), 'Account_item'] = 'Total NBV opening value'
aggregate_df.loc[(aggregate_df['MainCode'] == 'A14CY08') & (aggregate_df['SubCode'] == 'PPE0190'), 'Account_item'] = 'IT & Intan NBV closing value'
aggregate_df.loc[(aggregate_df['MainCode'] == 'A14CY08') & (aggregate_df['SubCode'] == 'PPE0010'), 'Account_item'] = 'IT & Intan NBV opening value'
aggregate_df.loc[(aggregate_df['MainCode'] == 'A13CY01') & (aggregate_df['SubCode'] == 'INT0190'), 'Account_item'] = 'IT & Intan NBV closing value'
aggregate_df.loc[(aggregate_df['MainCode'] == 'A13CY01') & (aggregate_df['SubCode'] == 'INT0010'), 'Account_item'] = 'IT & Intan NBV opening value'

In [ ]:
aggregate_df.head()

In [21]:
# Drop all the NaNs in the Account_item variable
aggregate_df = aggregate_df.dropna(subset=['Account_item'])

In [ ]:
aggregate_df.head()

We only need closing values per year and not opening values. The only reason we identified the opening values in the previous step was because the opening value for FY2017/18 is identical (expect where accountants have made adjustments) as the closing value for FY2016/17. By assuming this, we can add an extra year of closing asset values to the dataset which wasn't available from source. 

Use [np.where()](https://numpy.org/doc/stable/reference/generated/numpy.where.html) to change the Year to FY2016/17 wherethe Account_item is 'Total NBV opening value' or 'IT & Intan NBV opening value' and Year is FY2017/18, and change the Account_item from the opening value to closing values for rows which have been changed to FY2016/17.  

In [ ]:
# Create a row for closing values in FY2016/17 assuming they are equal to the opening values in FY2017/18

aggregate_df['Year'] = np.where((aggregate_df.Year == 'FY2017/18') & (aggregate_df.Account_item == 'IT & Intan NBV opening value'), 'FY2016/17', aggregate_df.Year)
aggregate_df['Account_item'] = np.where((aggregate_df.Year == 'FY2016/17') & (aggregate_df.Account_item == 'IT & Intan NBV opening value'), 'IT & Intan NBV closing value', aggregate_df.Account_item)
aggregate_df['Year'] = np.where((aggregate_df.Year == 'FY2017/18') & (aggregate_df.Account_item == 'Total NBV opening value'), 'FY2016/17', aggregate_df.Year)
aggregate_df['Account_item'] = np.where((aggregate_df.Year == 'FY2016/17') & (aggregate_df.Account_item == 'Total NBV opening value'), 'Total NBV closing value', aggregate_df.Account_item)

Now drop all opening values as you only kept them up until now to use as a proxy for the closing value in the year before the dataset started.

In [24]:
# Dropping opening value rows

aggregate_df = aggregate_df.drop(aggregate_df[aggregate_df.Account_item == 'IT & Intan NBV opening value'].index)
aggregate_df = aggregate_df.drop(aggregate_df[aggregate_df.Account_item == 'Total NBV opening value'].index)

There are multiple line items within the dataset with relate to components of each Account_item per provider and per year. The dataset needs to be collapsed on all the factor variables, and summing over the Value number in order to calculate the total value of each. [hint: use [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) and [sum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)]

In [27]:
# Task 7: Calculate the total value in Value number when grouping over 'Organisation Name', 'NHS code', 'Sector', 'Year', and 'Account_item'

aggregate_df = aggregate_df.groupby(['Organisation Name', 'NHS code', 'Sector', 'Year', 'Account_item'])[['Value number']].sum().reset_index()


# **4. Reshape wide**

While long format is the business for much of the data wrangling process, it is often useful to reshape into a wide format to make the output more user friendly / human readable, or to make coding of higher level analysis easier.

Use [pivot_table()](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) to reshape the dataframe so that each row has the IT & Intan NBV opening value and Total NBV opening value per provider per year.  

In [30]:
# Task 8: Reshape the data so that each row has the IT & Intan NBV opening value and Total NBV opening value per provider per year

aggregate_df = aggregate_df.pivot_table(index=['Organisation Name', 'NHS code', 'Sector', 'Year'],
                            columns='Account_item',
                            values='Value number').reset_index()

aggregate_df.head()

In [ ]:
# Create a new variable for the ratio of IT and Intangible assets to Total assets
aggregate_df['Tech_share'] = aggregate_df['IT & Intan NBV closing value']/aggregate_df['Total NBV closing value']

aggregate_df.head(40)